# Exploratory Data Analysis (EDA)

In this notebook, we will perform exploratory data analysis on the Tunisian stock market data to gain insights and identify patterns.


In [4]:
# Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Reading the data
data = pd.read_csv('./data/cleaned_weekly_stock_market.csv')

# Display the first few rows of the dataset
data.head()

,companyName,date,openingPrice,highestPrice,lowestPrice,closingPrice,volume
0,SOTUMAG,2014-06-16,1.76,1.80,1.73,1.73,44315.0
1,SOTUMAG,2014-06-23,1.78,1.82,1.77,1.82,9551.0
2,SOTUMAG,2014-06-30,1.81,1.82,1.81,1.81,2401.0
3,SOTUMAG,2014-07-07,1.78,1.82,1.78,1.80,10341.0
4,SOTUMAG,2014-07-14,1.83,1.83,1.76,1.81,27480.0


## Summary Statistics

We will start by generating summary statistics for the dataset to understand its basic properties.


In [6]:
# Summary statistics
summary_stats = data.describe()
summary_stats


,openingPrice,highestPrice,lowestPrice,closingPrice,volume
count,38631.000000,38631.000000,38631.000000,38631.000000,3.863100e+04
mean,106.641003,107.559123,105.715594,106.777993,2.821250e+05
std,791.642044,797.612176,785.735260,792.626816,2.504922e+06
min,0.110000,0.110000,0.110000,0.110000,0.000000e+00
25%,2.930000,2.990000,2.850000,2.930000,1.610000e+03
50%,6.690000,6.800000,6.530000,6.700000,1.003900e+04
75%,14.705000,14.910000,14.470000,14.710000,3.901750e+04
max,9592.970000,9670.220000,9578.670000,9663.860000,1.124223e+08
